In [2]:
import trimesh
import urdfpy
import numpy as np

In [26]:

# urdf_path = "urdf/RobotDog/RobotDog7kg.urdf"
urdf_path = "urdf/robotdog4kg/RobotDog_top-z-5_bottom-y-2.urdf"
urdf = urdfpy.URDF.load(urdf_path)

for link_name, link in urdf.link_map.items():
    print(link_name,link.inertial.mass)
np.sum([link.inertial.mass for link_name, link in urdf.link_map.items()])

base_link 1.5829483206719877
body_1_0_1 0.0651776633032798
body_1_1_1 0.08333511172191499
body_1_2_1 0.08054274287963308
body_1_3_1 0.06286681181080091
body_2_0_1 0.15435624844025742
body_2_1_1 0.1697787967564116
body_2_2_1 0.14940796074286172
body_2_3_1 0.17215466721428732


In [37]:
urdf_path = "urdf/RobotDog/RobotDog7kg.urdf"
urdf = urdfpy.URDF.load(urdf_path)

for link_name, link in urdf.link_map.items():
    print(link_name,link.inertial.mass,link)
np.sum([link.inertial.mass for link_name, link in urdf.link_map.items()])
scale = 0.32  # scale mass/inertia
for link_name, link in urdf.link_map.items():
    link.inertial.mass*=scale
    link.inertial.inertia*=scale
urdf.save("urdf/RobotDog/RobotDog3kg.urdf")

base_link 4.944006724025301 <urdfpy.urdf.Link object at 0x7f64d35f0e80>
body_1_1_1 0.17635098126357945 <urdfpy.urdf.Link object at 0x7f64d357b070>
body_1_2_1 0.21988900237553521 <urdfpy.urdf.Link object at 0x7f64d3633940>
body_1_3_1 0.2124436301601684 <urdfpy.urdf.Link object at 0x7f64d02df910>
body_1_4_1 0.16803507345033536 <urdfpy.urdf.Link object at 0x7f64d02e2c40>
body_2_1_1 0.4197478789995662 <urdfpy.urdf.Link object at 0x7f64d02df9d0>
body_2_2_1 0.4725136412635692 <urdfpy.urdf.Link object at 0x7f64d02e40d0>
body_2_3_1 0.39949807856162617 <urdfpy.urdf.Link object at 0x7f64d02e7e50>
body_2_4_1 0.47989575308955273 <urdfpy.urdf.Link object at 0x7f64d35f4df0>


2.5205683235414345